# Delete

We are going to see how to delete one or multiple rows of a table based on some filtering criteria

## Imports

In [ ]:
import psycopg2
import numpy as np
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
from pathlib import Path
import tifffile as tiff

## Connection

In [ ]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

## Loading the table from where we want to delete to check

In [ ]:
table_name = 'measurements'

data = qrs.get_data_metadata(table_name)

data

## Filtering to decide what to delete

We want to delete all the measurements with the metadata key signal_type_measurement as amplitude.

In [ ]:
filtered_data = data[data['aligned_measurement']]

filtered_data

## Deleting query

Now that we have only the rows that meet our deleting conditions, we can delete the measurements and their metadata with their ids

In [ ]:
indexes = filtered_data['id_measurement'].values.tolist() #get the indexes to delete

# Convert indexes to a tuple for SQL query
if len(indexes) == 0:
    print("No indexes to delete.")
elif len(indexes) == 1:
    indexes = tuple((indexes[0],indexes[0]))  # Ensure tuple format for single index
else:
    indexes = tuple(indexes)


sql = f'delete from {table_name} where {table_name}.id in {indexes}'
print(sql)

# Add RESTART IDENTITY command
reset_sql = f'ALTER TABLE {table_name} RESTART IDENTITY'
print(f"Reset identity SQL: {reset_sql}")

In [ ]:
# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL statement
cursor.execute(sql)

cursor.execute('COMMIT')

cursor.close()

## Check

If you execute the whole code again and the filtered table is empty, the delete whas succesfull.